In [ ]:
import argparse
import geopandas as gpd
import logging
import numpy as np
import os
import os.path as osp
import pandas as pd
import warnings
from shapely.geometry import box
from pyproj import Geod
from matplotlib import pyplot as plt
import sys 
sys.path.append('/mnt/c/Users/tandon/OneDrive - TomTom/Desktop/tomtom/Workspace/01_Rooftop_accuracy/APT-Realignment/src/apt_realignment')
from utils.geometric_utils import read_vector_data, create_logger, get_nearest_poly , prep_polygons_asarr,download_osm_building_footprint
from utils.haversine_distance import get_distance

In [ ]:
geod = Geod(ellps="WGS84")
warnings.filterwarnings("ignore")

from os import stat
from os.path import isfile

data_path  = "/mnt/c/Users/tandon/OneDrive - TomTom/Desktop/tomtom/Workspace/01_Rooftop_accuracy/BFP_Analysis_USA/data/data"
state = "CT"
city = "hartford"
parcel_dir = "Parcels_09003"

apt_data_path = os.path.join(data_path,state,"APT_2022_09_000_nam_usa_utx.shp" )
parcel_path = os.path.join(data_path,state,city,"{}/{}.shp".format(parcel_dir,parcel_dir))
msft_building = os.path.join(data_path,state,'Connecticut.geojson')
osm_building = os.path.join(data_path,state,city,'building_footprint.shp')

print("APT data path :",apt_data_path)
print("Parcel Data:",parcel_path)
print("Building_geojson: ",msft_building)
print("OSM building footprint: ",osm_building)

In [ ]:
def get_bfp_parcel_overlap(land_parcels,building_footprints):
    
    print("Reading Land Parcel data .. ")
    land_parcel_df = read_vector_data(land_parcels)
    print("Reading Building Footprints ..  ")
    footprint_df = read_vector_data(building_footprints)
    
    print("creating sjoin of land Parcel data and Building Footprints.. ")
    building_within_parcel_df = gpd.sjoin(land_parcel_df, footprint_df, op='intersects', how='left')
    building_within_parcel_df = building_within_parcel_df.dropna()  # drop columns with no Buildings

    def __get_buildingfootprint(val):
        return footprint_df['geometry'].loc[val]

    def __get_building_roi(data: gpd.GeoSeries):
        building_polygon = data['building_geometry']
        parcel_polygon = data['geometry']
        building_roi = None
        try:
            if building_polygon == np.nan:
                building_roi = parcel_polygon
            if building_polygon.area > parcel_polygon.area:
                building_roi = parcel_polygon.intersection(building_polygon)
            else:
                building_roi = building_polygon
        except:
            logging.error("error for {},{}".format(building_polygon, parcel_polygon))
        return building_roi

    building_within_parcel_df['building_geometry'] = building_within_parcel_df['index_right'].apply(lambda x: __get_buildingfootprint(x))

    building_within_parcel_df['building_roi'] = building_within_parcel_df.apply(lambda x: __get_building_roi(x), axis=1)
    building_within_parcel_df = building_within_parcel_df.drop(['index_right', 'release'],axis=1)
    building_within_parcel_df = building_within_parcel_df.dropna()
    return building_within_parcel_df

## Merge OSM and MSFT_building footprint

In [ ]:
if not os.path.isfile(osm_building):
    download_osm_building_footprint(county=city,state=state,out_path=osm_building)

In [28]:
def merge_osm_msft_data(msft_geojson,osm_geojson):
    osm_gdf = read_vector_data(osm_geojson)
    osm_bounds = box(*osm_gdf.total_bounds)

    msft_df = read_vector_data(msft_geojson)
    msft_df = msft_df.reset_index()

    osm_in_msft_df = gpd.sjoin(osm_gdf,msft_df, op='intersects', how='left')
    merged_data = osm_in_msft_df['geometry']

    def check_within_bounds(x):
        geo_polygon = None
        if osm_bounds.contains(x):
            geo_polygon = x
        return geo_polygon

    msft_geometries = msft_df.loc[~msft_df['index'].isin(osm_in_msft_df['index_right'].values)]['geometry']
    final_geometries = msft_geometries.apply(lambda x: check_within_bounds(x))
    final_gdf = merged_data.append(final_geometries[final_geometries.notna()])
    return final_gdf

merged_data = merge_osm_msft_data(msft_building,osm_building)
merged_data.to_file(os.path.join(data_path,state,city,'msft_osm_building_footprint.shp'), driver='ESRI Shapefile')

In [29]:
# final_data = merge_osm_msft_data(msft_building,county='bexar',state= 'Texas')
print(merged_data.shape)
merged_data.head()

(32389,)


0    POLYGON ((-72.68249 41.76437, -72.68242 41.764...
1    POLYGON ((-72.71554 41.79882, -72.71535 41.798...
2    POLYGON ((-72.71387 41.79993, -72.71392 41.799...
3    POLYGON ((-72.67338 41.76273, -72.67296 41.762...
4    POLYGON ((-72.67454 41.76341, -72.67445 41.763...
Name: geometry, dtype: geometry

## Process Land Parcels and Building footprints

This code read the parcel and building footprint data and finds out sjoin of both geometries . It rejects the data where building footprints is not found .

In [ ]:
parcel_bfp_df = get_bfp_parcel_overlap(land_parcels=parcel_path,building_footprints=msft_building)

In [ ]:
parcel_bfp_df.head(4)

## Get BFP within Parcel data 

In [ ]:
building_within_parcel_count = parcel_bfp_df.groupby('PRCLDMPID')['geometry'].count()
# building_within_parcel_count.hist(bins=np.arange(0,10,1))

fig, ax = plt.subplots(figsize=(17,9))

ax.set_title("Histogram of APT to centroid distance on/not on BFP")
ax.set_xlabel("APT point to centroid distance(meters)")
ax.set_ylabel("counts")

frqTrue, edgesTrue = np.histogram(building_within_parcel_count, bins = np.arange(1,10,1))
p1 = ax.bar(edgesTrue[:-1], frqTrue, width=np.diff(edgesTrue), edgecolor="black", align="edge",alpha=0.4,label='Address Points on Rooftop',color='orange')

plt.legend()
plt.show()

In [ ]:
def get_buildings_within_parcel(data: gpd.GeoSeries, count=None):
    print("Acquiring BFP's within land Parcels ".format(count))
    building_within_parcel_count = data.groupby('PRCLDMPID')['geometry'].count()
    if count == 1:
        parcel_ids_with_one_building = list(building_within_parcel_count[building_within_parcel_count == 1].keys())
        filtered_dataframe = data[data['PRCLDMPID'].isin(parcel_ids_with_one_building)]
    elif count == 2:
        parcel_ids_with_two_buildings = list(building_within_parcel_count[building_within_parcel_count == 2].keys())
        filtered_dataframe = data[data['PRCLDMPID'].isin(parcel_ids_with_two_buildings)]
    else:
        parcel_ids_with_n_buildings = list(building_within_parcel_count[building_within_parcel_count > 2].keys())
        filtered_dataframe = data[data['PRCLDMPID'].isin(parcel_ids_with_n_buildings)]
    return filtered_dataframe

df_parcel_within_bfp = get_buildings_within_parcel(parcel_bfp_df, count=2)

In [ ]:
print("Data shape",df_parcel_within_bfp.shape)
df_parcel_within_bfp.head(6)

In [ ]:
df_parcel_within_bfp = df_parcel_within_bfp.drop(['index_right','release'],axis=1)

## Read Anchor point data

In [ ]:
def get_parcel_anchorpoints(anchor_points_data,input_dataframe: gpd.GeoSeries):
    print("Reading Anchor-Points data over Parcel-Building Geo-Dataframe")
    
    anchorpoint_df = read_vector_data(anchor_points_data)
    apt_df_columns = list(anchorpoint_df.columns)
    
    print("Processing Anchor-Points and Parcel-Building Geo-Dataframe")
    # find spatial join of input_dataframe with anchorpoint
    grouped_df = gpd.sjoin(input_dataframe, anchorpoint_df, op='contains', how='inner')
    print(grouped_df.keys())

    def _get_apt_point(val):
        return anchorpoint_df['geometry'].loc[val]

    grouped_df['APT'] = grouped_df['index_right'].apply(lambda x: _get_apt_point(x))
    grouped_df = grouped_df.drop(['index_right'], axis=1)
    return grouped_df , apt_df_columns

process_df , cols = get_parcel_anchorpoints(apt_data_path,df_parcel_within_bfp)

In [ ]:
print("Required columns",cols)
process_df.head(6)

## Start Processing

In [ ]:
def check_point_on_polygon(bfp_polygon, apt_point):
    flag = False
    if bfp_polygon.contains(apt_point):
        flag = True
    return flag

def multi_map(x,required_cols, area_thresh=150,bfp_per_parcel =2 ):
    ret = dict()
    req_columns = ['PRCLDMPID', 'building_roi', 'APT'] + required_cols
    # get two buildings as we have selected 
    
    
    for cols in req_columns:
        ret[cols] = x.iloc[0][cols]
    
    if bfp_per_parcel ==2:
        building_polygons = list(x['building_roi'][:2])
        geo_area = list(x['building_roi'].apply(lambda poly: abs(geod.geometry_area_perimeter(poly)[0])))[:bfp_per_parcel]
        area_diff = geo_area[0] - geo_area[1]
        
        if area_diff > 0:
            if area_diff > area_thresh:
                ret['building_roi'] = building_polygons[0]
            else:
                ret['building_roi'] = (list(x['building_roi'][:bfp_per_parcel])[get_nearest_poly(list(x['APT'])[0], building_polygons)])

        elif area_diff <= 0:
            if np.abs(area_diff) > area_thresh:
                ret['building_roi'] = building_polygons[1]
            else:
                ret['building_roi'] = (list(x['building_roi'][:bfp_per_parcel])[get_nearest_poly(list(x['APT'])[0], building_polygons)])
    
    if bfp_per_parcel>2:
        building_polygons = list(x['building_roi'][:3])
        mnr_apt = list(x['APT'])[0]
        point_on_polygon = False
        
        for polygon in building_polygons:
            if check_point_on_polygon(polygon,mnr_apt):
                point_on_polygon=True 
        if not point_on_polygon:
            ret['building_roi'] = (list(x['building_roi'][:3])[get_nearest_poly(mnr_apt, building_polygons)])
    return ret

In [ ]:
def get_apt_to_bfp_distance(self, data):
    anchor_point = data['APT']
    bfp_centroid = data['updated_geometries']
    return get_distance(anchor_point, bfp_centroid)

complexity = 2 

if complexity ==1 :
    process_df['updated_geometries'] = process_df['building_roi'].apply(lambda x: x.centroid)
    process_df['updated_dt'] = process_df.apply(lambda x: get_apt_to_bfp_distance(x),axis=1)

# if complexity ==2 :
#     process_df = process_df.groupby(['PRCLDMPID'], as_index=False).apply(lambda x: pd.Series(multi_map(x,required_cols=cols)))

filter_df = process_df.groupby(['PRCLDMPID'], as_index=False).apply(lambda x: pd.Series(multi_map(x,required_cols=cols,bfp_per_parcel=3)))

In [ ]:
print(filter_df.shape)
filter_df.head()